-> Descobri por que o regex para tirar numeros e caracteres especiis não está titando os números

# Bibliotecas

In [129]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from unidecode import unidecode

nltk.download('punkt')  # Faz o download do pacote de dados necessário
nltk.download('stopwords')  # Faz o download das stopwords



[nltk_data] Downloading package punkt to /home/marina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/marina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Funções

In [158]:
def contar_termos(arquivo, coluna=None, type='txt'):
    if type == 'txt':
        with open(arquivo, 'r', encoding='utf-8') as file:
            texto = file.read()
        
            # Tokeniza as palavras no texto
            tokens = word_tokenize(texto)
        palavras = [word.lower() for word in tokens]

    elif type == 'csv':
        df = pd.read_csv(arquivo, usecols=[coluna])
        df = df.drop_duplicates().reset_index(drop=True)
        # Tokeniza as palavras na coluna de texto
        tokens = df[coluna].apply(word_tokenize)
        palavras = [word.lower() for sublist in tokens for word in sublist]
    
    # Remove stopwords e caracteres especiais/emojis da lista de palavras
    stopwords_list = set(stopwords.words('portuguese'))  # Altere para o idioma desejado, se necessário
    palavras = [unidecode(word) for word in palavras if word not in stopwords_list]
    palavras = [re.sub(r'[^a-zA-Z0-9]', '', word) for word in palavras if word not in stopwords_list]
    
    # Remove palavras vazias
    palavras = [word for word in palavras if word]
    
    # Cria bigramas (termos compostos) e conta a ocorrência de cada termo
    bigramas = list(nltk.bigrams(palavras))
    termos = [f'{word1} {word2}' for word1, word2 in bigramas]
    contagem = pd.Series(termos).value_counts().reset_index()
    contagem.columns = ['Termo', 'Quantidade']
    
    return contagem


In [159]:
def contar_palavras(arquivo, coluna=None, type='txt'):
    if type == 'txt':
        with open(arquivo, 'r', encoding='utf-8') as file:
            texto = file.read()
        
            # Tokeniza as palavras no texto
            tokens = word_tokenize(texto)
        palavras = [word.lower() for word in tokens]

    elif type == 'csv':
        df = pd.read_csv(arquivo, usecols=[coluna])
        df = df.drop_duplicates().reset_index(drop=True)
        # Tokeniza as palavras na coluna de texto
        tokens = df[coluna].apply(word_tokenize)
        palavras = [word.lower() for sublist in tokens for word in sublist]
    
    # Remove as stopwords da lista de palavras
    stopwords_list = set(stopwords.words('portuguese'))  # Altere para o idioma desejado, se necessário
    palavras = [unidecode(word) for word in palavras if word not in stopwords_list]
    palavras = [re.sub(r'[^a-zA-Z0-9]', '', word) for word in palavras if word not in stopwords_list]
    
    # Remove palavras vazias
    palavras = [word for word in palavras if word]
    
    # Conta a ocorrência de cada palavra
    contagem = pd.Series(palavras).value_counts().reset_index()
    contagem.columns = ['Palavra', 'Quantidade']
    
    return contagem


# Variáveis Globais

In [160]:
path = '../data/'
insta = path+'cnnviagemegastronomia.txt'
reddit = path+'translated_top_posts.csv'

# Palavras

In [161]:
resultados = list()
resultados.append(contar_palavras(insta))

coluns_reddit = ['Title', 'Post Text', 'Comments']
for col in coluns_reddit:
    resultados.append(contar_palavras(reddit, col, 'csv'))

df_resultados = pd.concat(resultados)
df_resultados_agg = df_resultados.groupby('Palavra', as_index=False).sum()
df_resultados_agg = df_resultados_agg.sort_values(by='Quantidade', ascending=False)

In [162]:
len(df_resultados_agg)

36369

In [163]:
df_resultados_agg.head(15)

,Palavra,Quantidade
35111,vinho,6483
35164,vinhos,3538
15987,garrafa,2846
26634,pode,2249
16014,garrafas,1517
2628,alguns,1465
5510,bom,1441
24569,obrigado,1395
12830,entao,1355
3343,apenas,1296


# Termos

In [164]:
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob

# download de stopwords de nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/marina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/marina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [165]:
resultados = list()
resultados.append(contar_termos(insta))

coluns_reddit = ['Title', 'Post Text', 'Comments']
for col in coluns_reddit:
    resultados.append(contar_termos(reddit, col, 'csv'))

df_resultados = pd.concat(resultados)
df_resultados_agg = df_resultados.groupby('Termo', as_index=False).sum()
df_resultados_agg = df_resultados_agg.sort_values(by='Quantidade', ascending=False)

In [166]:
len(df_resultados_agg)

246921

In [167]:
df_resultados_agg.head(15)

,Termo,Quantidade
108621,garrafa vinho,561
155560,notas degustacao,520
174928,podem ter,477
71531,deste subreddit,455
238876,vinho https,455
104259,foto garrafa,455
17197,alguma duvida,454
119838,inclua notas,453
189308,quanto vale,453
170407,perguntar quanto,453
